# Registration 

In [1]:
%matplotlib inline

import os
from helpers import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.3.8
SimpleITK version = 2.3.0


In [35]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
print(f'project folder = {BASE_DIR}')

project folder = D:\NEURO_DATA\Antspy\MRI-preprocessing-techniques


In [36]:
raw_examples = [
    'CAHI01.nii.gz']

### Showing Rwa image

In [37]:
raw_example = raw_examples[0]
raw_img_path = os.path.join(BASE_DIR, 'assets',  'raw_examples',raw_example) #'assets',  'raw_examples'
raw_img_ants = ants.image_read(raw_img_path, reorient='IAL')

explore_3D_array(arr=raw_img_ants.numpy())

interactive(children=(IntSlider(value=127, description='SLICE', max=255), Output()), _dom_classes=('widget-int…

# Template Image

In [39]:
# here we use MNI 152 template as a reference point
template_img_path = os.path.join(BASE_DIR, 'assets', 'templates', 'mni_icbm152_t1_tal_nlin_sym_09a.nii')
MNI1_template_img_ants = ants.image_read(template_img_path, reorient='IAL')

explore_3D_array(arr = MNI1_template_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [40]:
print('\t\tRAW IMG')
print(raw_img_ants)

print('\t\tTEMPLATE IMG')
print(MNI1_template_img_ants)

		RAW IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (256, 256, 192)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (191.0, 0.0, 0.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]

		TEMPLATE IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]



# Perform Registrations 

In [42]:
# “SyN”: Symmetric normalization: Affine + deformable transformation,
# with mutual information as optimization metric.

transformation_MNI1 = ants.registration(
    fixed=MNI1_template_img_ants,
    moving=raw_img_ants, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [000001CDFF5145A0,000001CDFF514540,1] -m mattes[000001CDFF5145A0,000001CDFF514540,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[000001CDFF5145A0,000001CDFF514540,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [C:\Users\panka\AppData\Local\Temp\tmp3oronuwy,000001CDFF514590,000001CDFF514570] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1


In [43]:
print(transformation_MNI1)

{'warpedmovout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'warpedfixout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (256, 256, 192)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (191.0, 0.0, 0.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'fwdtransforms': ['C:\\Users\\panka\\AppData\\Local\\Temp\\tmp3oronuwy1Warp.nii.gz', 'C:\\Users\\panka\\AppData\\Local\\Temp\\tmp3oronuwy0GenericAffine.mat'], 'invtransforms': ['C:\\Users\\panka\\AppData\\Local\\Temp\\tmp3oronuwy0GenericAffine.mat', 'C:\\Users\\panka\\AppData\\Local\\Temp\\tmp3oronuwy1InverseWarp.nii.gz']}


In [44]:
MNI_registered_img_ants = transformation_MNI1['warpedmovout'] # warpedmovout: Moving image warped to space of fixed image.

explore_3D_array(arr=MNI_registered_img_ants.numpy())

# Image after Registrations

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [45]:
out_folder =  os.path.join(BASE_DIR, 'assets', 'preprocessed')
out_folder = os.path.join(out_folder, raw_example.split('.')[0]) # create folder with name of the raw file
os.makedirs(out_folder, exist_ok=True) # create folder if not exists

out_filename = add_suffix_to_filename(raw_example, suffix='MNI_registered')
out_path = os.path.join(out_folder, out_filename)

print(raw_img_path[len(BASE_DIR):])
print(out_path[len(BASE_DIR):])

\assets\raw_examples\CAHI01.nii.gz
\assets\preprocessed\CAHI01\CAHI01_MNI_registered.nii.gz


### Using MASK

In [46]:
#Mask
mask_example = add_suffix_to_filename(raw_example,suffix='M') #suffix='mask' CAHI01M.ch.nii.gz
mask_img_path = os.path.join(BASE_DIR, 'assets', 'raw_examples', mask_example)
mask_img_ants = ants.image_read(mask_img_path, reorient='IAL')

explore_3D_array_with_mask_contour(
    arr=raw_img_ants.numpy(),
    mask=mask_img_ants.numpy()
)

interactive(children=(IntSlider(value=127, description='SLICE', max=255), Output()), _dom_classes=('widget-int…

In [47]:
MNI_template_img_path = os.path.join(BASE_DIR, 'assets', 'templates', 'mni_icbm152_t1_tal_nlin_sym_09a.nii')
MNI_template_img_ants = ants.image_read(MNI_template_img_path, reorient='IAL')

explore_3D_array(arr = MNI_template_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [28]:
print('\t\tRAW IMG')
print(raw_img_ants)

print('\t\tTEMPLATE IMG')
print(MNI_template_img_ants)

		RAW IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (256, 256, 192)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (191.0, 0.0, 0.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]

		TEMPLATE IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]



In [29]:
#perform registrations
transformation = ants.registration(
    fixed=MNI_template_img_ants,
    moving=raw_img_ants, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [000001CD8652DB90,000001CD8652DC30,1] -m mattes[000001CD8652DB90,000001CD8652DC30,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[000001CD8652DB90,000001CD8652DC30,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [C:\Users\panka\AppData\Local\Temp\tmpulelae_5,000001CD8652DA70,000001CD8652DBC0] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1


In [30]:
registered_img_ants = transformation['warpedmovout']

explore_3D_array(arr=registered_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

### Move Raw Mask to Native space

In [49]:
registered_mask_img_ants = ants.apply_transforms(
    moving=mask_img_ants,
    fixed=transformation_MNI1['warpedmovout'],
    transformlist=transformation_MNI1['fwdtransforms'],
    verbose=True
)

['-d', '3', '-i', '000001CD8FC29CE0', '-o', '000001CD8FC29D00', '-r', '000001CD8FC29D80', '-n', 'linear', '-t', 'C:\\Users\\panka\\AppData\\Local\\Temp\\tmp3oronuwy1Warp.nii.gz', '-t', 'C:\\Users\\panka\\AppData\\Local\\Temp\\tmp3oronuwy0GenericAffine.mat']


In [50]:
explore_3D_array_with_mask_contour(
    arr=registered_img_ants.numpy(),
    mask=registered_mask_img_ants.numpy()
)

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

### Image Registrations task based on Talairachatlas template

In [20]:
# here we using Talairach template as a reference point
template_img_path = os.path.join(BASE_DIR, 'assets', 'templates', 'talairach.nii')
Talairach_template_img_ants = ants.image_read(template_img_path, reorient='LAI')

explore_3D_array(arr = template_img_ants.numpy())

interactive(children=(IntSlider(value=54, description='SLICE', max=109), Output()), _dom_classes=('widget-inte…

In [21]:
print('\t\tRAW IMG')
print(raw_img_ants)

print('\t\tTEMPLATE IMG')
print(Talairach_template_img_ants)

		RAW IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (256, 256, 192)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (191.0, 0.0, 0.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]

		TEMPLATE IMG
ANTsImage (LAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (141, 172, 110)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (70.0, -69.0, -42.0)
	 Direction  : [-1.  0.  0.  0.  1.  0.  0.  0.  1.]



## Perform Registration using Talairach Template

In [22]:
Tal_transformation = ants.registration(
    fixed=Talairach_template_img_ants,
    moving=raw_img_ants, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [000001CDE73A8AB0,000001CDE73A8D40,1] -m mattes[000001CDE73A8AB0,000001CDE73A8D40,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[000001CDE73A8AB0,000001CDE73A8D40,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [C:\Users\panka\AppData\Local\Temp\tmpv4llvky5,000001CDE73A89F0,000001CDE73A8B80] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1


### Observed dimentaions after registrations

In [23]:
# warpedmovout: Moving image warped to space of fixed image.
# warpedfixout: Fixed image warped to space of moving image.
# fwdtransforms: Transforms to move from moving to fixed image. 
# invtransforms: Transforms to move from fixed to moving image.
print(Tal_transformation)

{'warpedmovout': ANTsImage (LAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (141, 172, 110)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (70.0, -69.0, -42.0)
	 Direction  : [-1.  0.  0.  0.  1.  0.  0.  0.  1.]
, 'warpedfixout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (256, 256, 192)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (191.0, 0.0, 0.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'fwdtransforms': ['C:\\Users\\panka\\AppData\\Local\\Temp\\tmpv4llvky51Warp.nii.gz', 'C:\\Users\\panka\\AppData\\Local\\Temp\\tmpv4llvky50GenericAffine.mat'], 'invtransforms': ['C:\\Users\\panka\\AppData\\Local\\Temp\\tmpv4llvky50GenericAffine.mat', 'C:\\Users\\panka\\AppData\\Local\\Temp\\tmpv4llvky51InverseWarp.nii.gz']}


In [24]:
Tal_registered_img_ants = Tal_transformation['warpedmovout'] # warpedmovout: Moving image warped to space of fixed image.

explore_3D_array(arr=Tal_registered_img_ants.numpy())

interactive(children=(IntSlider(value=70, description='SLICE', max=140), Output()), _dom_classes=('widget-inte…

In [18]:
explore_3D_array(arr=MNI_registered_img_ants.numpy())

interactive(children=(IntSlider(value=54, description='SLICE', max=109), Output()), _dom_classes=('widget-inte…

In [51]:
Tal_registered_mask_img_ants = ants.apply_transforms(
    moving=mask_img_ants,
    fixed=Tal_transformation['warpedmovout'],
    transformlist=Tal_transformation['fwdtransforms'],
    verbose=True
)

['-d', '3', '-i', '000001CD86238FE0', '-o', '000001CD86238EF0', '-r', '000001CD86239000', '-n', 'linear', '-t', 'C:\\Users\\panka\\AppData\\Local\\Temp\\tmpv4llvky51Warp.nii.gz', '-t', 'C:\\Users\\panka\\AppData\\Local\\Temp\\tmpv4llvky50GenericAffine.mat']


In [53]:
explore_3D_array_with_mask_contour(
    arr=Tal_registered_img_ants.numpy(),
    mask=Tal_registered_mask_img_ants.numpy()
)

interactive(children=(IntSlider(value=70, description='SLICE', max=140), Output()), _dom_classes=('widget-inte…

In [54]:
1497/6

249.5

In [56]:
1497+499

1996

In [57]:
1497-249.5


1247.5

In [58]:
1247/5

249.4

In [59]:
750/6

125.0

In [60]:
249.5+125

374.5

In [61]:
711/6

118.5

In [62]:
249.5+118.4

367.9

In [63]:
750/6

125.0

In [64]:
249.5+125

374.5